# Project

Welcome to the group project! The project is based on the [ACM RecSys 2021 Challenge](https://recsys-twitter.com/).

- Detailed information about the task, submission and grading can be found in a [dedicates site on TUWEL](https://tuwel.tuwien.ac.at/mod/page/view.php?id=1217340).
- Information about the dataset structure [on this site on TUWEL](https://tuwel.tuwien.ac.at/mod/page/view.php?id=1218810).

In [1]:
team_name = "team_5" # your team name e.g. 'team_1'
team_members = [("Simone Andreetto","01635069"),
                ("Adrian Bracher",""),
                ("Dominik Mailer","01634043"),
                ("Andreas Merckel",""),
                ("Dominik Pülke","12019262"),
                ("Sebastian Scholz","01526884"),
                ("Felix Winterleitner","01612776"),
                ("Ahmadou Wagne","12002293")] # [("Jane Doe","012345678"), ("John Doe","012345678")]


In [2]:
print(team_name)
print(team_members)

team_5
[('Simone Andreetto', '01635069'), ('Adrian Bracher', ''), ('Dominik Mailer', '01634043'), ('Andreas Merckel', ''), ('Dominik Pülke', '12019262'), ('Sebastian Scholz', '01526884'), ('Felix Winterleitner', '01612776'), ('Ahmadou Wagne', '12002293')]


In [3]:
path_to_data = '../shared/data/project/training/'
dataset_type = 'one_hour' # all_sorted, one_day, one_hour, one_week

In [4]:
import os
import re
import csv
import datetime
import model_nn
import pandas as pd


from dataprep import import_data

In [5]:


all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager"]

all_features_to_idx = dict(zip(all_features, range(len(all_features))))

def parse_input_line(line):
    features = line #.split("\x01")
    tweet_id = features[all_features_to_idx['tweet_id']]
    user_id = features[all_features_to_idx['engaging_user_id']]
    input_feats = features[all_features_to_idx['text_tokens']]
    tweet_timestamp = features[all_features_to_idx['tweet_timestamp']]
    return tweet_id, user_id, input_feats, tweet_timestamp


def evaluate_test_set():
    expanded_path = os.path.expanduser(path_to_data)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
        
    with open('results.csv', 'w') as output:
        for file in part_files:
            with open(file, 'r') as f:
                #df = pd.read_csv(f, sep="\x01")
                #print(f"df: {df}")
                linereader = csv.reader(f, delimiter='\x01')
                last_timestamp = None
                for row in linereader:
                    tweet_id, user_id, features, tweet_timestamp = parse_input_line(row)   
                    reply_pred = reply_pred_model(features) # reply_model
                    #retweet_pred = retweet_pred_model(features) # retweet_model
                    #quote_pred = quote_pred_model(features) # pred_model
                    #fav_pred = fav_pred_model(features) # fav_model
                    
                    print(features)
                    #reply_pred = model_nn.reply_pred_model(features)
                    #retweet_pred = model_nn.retweet_pred_model(features)
                    #quote_pred = model_nn.quote_pred_model(features)
                    #fav_pred = model_nn.fav_pred_model(features)
                    
                    
                    # print(str(tweet_timestamp))
                    # print(str(reply_pred)+" "+str(retweet_pred)+" "+str(quote_pred)+" "+str(fav_pred))
                    
                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')


In [6]:
evaluate_test_set()

NameError: name 'reply_pred_model' is not defined

# NN-Model

In [7]:
import os
import re
import csv
import datetime
import model_nn
import pandas as pd

from dataprep import import_data
import dataprep
from sklearn.preprocessing import StandardScaler

In [20]:
used_features = [
            # "text_tokens",
            "hashtags",
            # "tweet_id",
            "present_media",
            "present_links",
            "present_domains",
            "tweet_type",
            # "language",
            # "tweet_timestamp",
            # "engaged_with_user_id",
            # "engaged_with_user_follower_count",
            "engaged_with_user_following_count",
            "engaged_with_user_is_verified",
            "engaged_with_user_account_creation",
            # "engaging_user_id",
            # "enaging_user_follower_count",
            # "enaging_user_following_count",
            # "enaging_user_is_verified",
            # "enaging_user_account_creation",
            "engagee_follows_engager"
    ]

all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager"]

all_features_to_idx = dict(zip(all_features, range(len(all_features))))

def parse_input_line(line):
        
    features = line #.split("\x01")
    
    hashtags = features[all_features_to_idx['hashtags']]
    present_media = features[all_features_to_idx['present_media']]
    present_links = features[all_features_to_idx['present_links']]
    present_domains = features[all_features_to_idx['present_domains']]
    tweet_type = features[all_features_to_idx['tweet_type']]
    engaged_with_user_following_count = features[all_features_to_idx['engaged_with_user_following_count']]
    engaged_with_user_is_verified = features[all_features_to_idx['engaged_with_user_is_verified']]
    engaged_with_user_account_creation = features[all_features_to_idx['engaged_with_user_account_creation']]
    engagee_follows_engager = features[all_features_to_idx['engagee_follows_engager']]
    
    return (hashtags, present_media, present_links, present_domains, tweet_type, engaged_with_user_following_count, engaged_with_user_is_verified, engaged_with_user_account_creation, engagee_follows_engager)


def evaluate_test_set():
    expanded_path = os.path.expanduser(path_to_data)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
        
    with open('results.csv', 'w') as output:
        for file in part_files:
            with open(file, 'r') as f:
                linereader = csv.reader(f, delimiter='\x01')
                last_timestamp = None
                df = pd.DataFrame(columns=all_features)
                i = 0
                for row in linereader:
                    df.loc[i] = row[:20]
                    i += 1
                    if i > 10000:
                        break
                df_complete = df.copy()
                df = df.loc[:, used_features]
                df = dataprep.transform_data(df)
                scale = StandardScaler()
                #df = scale.fit_transform(df)
                #df = pd.DataFrame(columns=all_features, data=df)
                df = pd.DataFrame(scale.fit_transform(df.values), columns=df.columns, index=df.index)
                for index, row in df.iterrows():
                    tweet_id = df_complete.iloc[[index]]["tweet_id"]
                    user_id = df_complete.iloc[[index]]["engaging_user_id"]
                    #feature_tuple = parse_input_line(row)   
                    #reply_pred = reply_pred_model(features) # reply_model
                    #retweet_pred = retweet_pred_model(features) # retweet_model
                    #quote_pred = quote_pred_model(features) # pred_model
                    #fav_pred = fav_pred_model(features) # fav_model
                    
                    #print(feature_tuple)
                    #print(df.iloc[[index]]["engaged_with_user_following_count"])
                    reply_pred = model_nn.reply_pred_model(df.iloc[[index]])
                    retweet_pred = model_nn.retweet_pred_model(df.iloc[[index]])
                    quote_pred = model_nn.quote_pred_model(df.iloc[[index]])
                    fav_pred = model_nn.fav_pred_model(df.iloc[[index]])
                    
                    
                    # print(str(tweet_timestamp))
                    #print(str(reply_pred)+" "+str(retweet_pred)+" "+str(quote_pred)+" "+str(fav_pred))
                    output.write(f'{tweet_id.values[0]},{user_id.values[0]},{reply_pred[0]},{retweet_pred[0]},{quote_pred[0]},{fav_pred[0]}\n')


In [ ]:
evaluate_test_set()

395A05A1E8A0A4CEB2E623281C7A41EE,736278C2FEC488516CDA4ED6952A2154,0.07350903381381661,0.0,0.0,0.4311371331059942
81E8247F4E74A0FCDBA911E1A3CB5412,19D5367D835484236CAF9DBEF475FF7A,0.4111913939770835,0.24739606864651675,0.1601019370885886,0.0
81E8247F4E74A0FCDBA911E1A3CB5412,40BEB04CF8D3CB02449879668656FFDB,0.4111913939770835,0.24739606864651675,0.1601019370885886,0.0
81E8247F4E74A0FCDBA911E1A3CB5412,6415C94D3C27BA84C069DE049EBB3EDE,0.4111913939770835,0.24739606864651675,0.1601019370885886,0.0
81E8247F4E74A0FCDBA911E1A3CB5412,7E614D5881BC18768880CC374C4BE821,0.4111913939770835,0.24739606864651675,0.1601019370885886,0.0
81E8247F4E74A0FCDBA911E1A3CB5412,9CFE74AF494F669C35496B000C5843AB,0.4111913939770835,0.24739606864651675,0.1601019370885886,0.0
81E8247F4E74A0FCDBA911E1A3CB5412,B5FAB4F5EC1B12BE7E7FE0C27662EB80,0.094573174154505,0.0719135942232225,0.057732461282024694,0.0
40729594A0EC3DB7D690C6789EC11992,BDAFE5407EF4AA3E63E12B8074CA6951,0.4213816382618901,0.0,0.0,0.41507648916572365
E5DE59

68470D7148500EFAC6B9535F5ACF4E4C,1E883F969FEF3C304CB430FC18D3D438,0.0,0.0,0.0,1.0
68470D7148500EFAC6B9535F5ACF4E4C,315A55AEB1DE6A2EA9C936B013E639F2,0.0,0.0,0.0,1.0
68470D7148500EFAC6B9535F5ACF4E4C,3167851AF1C7D0C185AB48628B03F902,0.0,0.0,0.0,1.0
68470D7148500EFAC6B9535F5ACF4E4C,32ECE018EEE605CE1A11C8453931727B,0.0,0.0,0.0,1.0
68470D7148500EFAC6B9535F5ACF4E4C,4682A80DF1F8F860672B098A5C26BD57,0.0,0.0,0.0,1.0
68470D7148500EFAC6B9535F5ACF4E4C,4C01B2658DBEA4DF80E9BA9F30739FAB,0.0,0.0,0.0,1.0
68470D7148500EFAC6B9535F5ACF4E4C,4E490DEE62EC1880CD76047C35542D8A,0.0,0.0,0.0,1.0
68470D7148500EFAC6B9535F5ACF4E4C,5E85C9B96CE880F195C7478C50D0DA75,0.0,0.0,0.0,1.0
68470D7148500EFAC6B9535F5ACF4E4C,80C87D432B3C25DFDA5904FA30A10FF2,0.0,0.0,0.0,1.0
68470D7148500EFAC6B9535F5ACF4E4C,88D1DA7307AD4566320462602D49A559,0.0,0.0,0.0,1.0
68470D7148500EFAC6B9535F5ACF4E4C,9338F033284295F28592B186316FDC93,0.0,0.0,0.0,1.0
68470D7148500EFAC6B9535F5ACF4E4C,990DDE7CB7A87DBFEB038FD880FFA425,0.0,0.0,0.0,1.0
68470D7148500EFA

AA31EE4BDF29C959505B22FC99FC42F9,83AAF68565984746921D59FE39581466,0.0,0.0,0.011742574167694919,0.0
AA31EE4BDF29C959505B22FC99FC42F9,926498A5583A201218FA83D3238EAE31,0.0,0.0,0.011742574167694919,0.0
A0C18744FD1F64249143A389C6848E2D,1C195D237DE1F9E5D332E38E6AFCD7AB,0.2907316167493326,0.0,0.33156088739458817,1.0
A0C18744FD1F64249143A389C6848E2D,5DF74B3CC2B51046F1B2D0C390464767,0.41042796353796274,0.0,0.3798313037467587,1.0
B41FA8D5632B7A25F407EE1F5B7440F2,727B57ED098C85128F457EB24552EA0D,0.6971668758316775,0.0,1.0,0.4371933365618036
B41FA8D5632B7A25F407EE1F5B7440F2,F2A7A1B980EDE35AFA0D7201B54C6F5A,0.6971668758316775,0.0,1.0,0.4371933365618036
E0C7D81140B16F58B1B005228A29E0B3,5E468DFB73C4D64A8A8EEB6E7AF92E9B,0.13062960001820847,0.0,0.48372040521712023,1.0
E0C7D81140B16F58B1B005228A29E0B3,C7DF61A61BC7F785A4F67AF31FFE4C9B,0.13062960001820847,0.0,0.48372040521712023,1.0
A0AC380D8F798C06EEFDC6D32EBCC409,55F9B83571B2167C38392137E870FA20,0.0,0.0,0.0,1.0
A8AC72AD1B865DD430512FBDE6E2FA4B,04805AAA7

B750B9F07B7C364B5321C9D80279B137,186F4D5420C81B38FE246F155A151BC6,0.0,0.0,0.02382369988144445,0.0
B750B9F07B7C364B5321C9D80279B137,1AB31E09CD06E6FBDD11B33345E957FC,0.0,0.0,0.02382369988144445,0.0
B750B9F07B7C364B5321C9D80279B137,2596EAFCE3FC7A32945C85CA97CB7303,0.0,0.0,0.02382369988144445,0.0
B750B9F07B7C364B5321C9D80279B137,2EA50B4314B1F9A785659AEC8380F5F4,0.0,0.0,0.02382369988144445,0.0
B750B9F07B7C364B5321C9D80279B137,3D206BE182189F6CD7206864CB6D2B21,0.0,0.0,0.02382369988144445,0.0
B750B9F07B7C364B5321C9D80279B137,40C4DC3B1E785C9F0C23795CC803F3E5,0.0,0.0,0.02382369988144445,0.0
B750B9F07B7C364B5321C9D80279B137,4ED629D0547F51A49EC860ED74EEE2A0,0.0,0.0,0.02382369988144445,0.0
B750B9F07B7C364B5321C9D80279B137,4F8B0741C84A663C77A1452B396757F0,0.0,0.0,0.02382369988144445,0.0
B750B9F07B7C364B5321C9D80279B137,54BBD49D92702BFE236E864D5F7D83D0,0.0,0.0,0.02382369988144445,0.0
B750B9F07B7C364B5321C9D80279B137,56A6F62DA0294E5224217F0C9D511D1E,0.0,0.0,0.02382369988144445,0.0
B750B9F07B7C364B5321

64153475F8549F693DC486E8E200F329,61015C883A9C79DF6E142F7188530223,0.7058200262904907,0.16101740204962262,0.45211401975677207,0.0
64153475F8549F693DC486E8E200F329,62AC1EE25D8BD861CA2122C314DD0F7F,0.7058200262904907,0.16101740204962262,0.45211401975677207,0.0
64153475F8549F693DC486E8E200F329,649E8ECE76557E8A1E315E451A5A062B,0.7058200262904907,0.16101740204962262,0.45211401975677207,0.0
64153475F8549F693DC486E8E200F329,65AF0CBF05FAC0F2D3081E364F345E6B,0.7058200262904907,0.16101740204962262,0.45211401975677207,0.0
64153475F8549F693DC486E8E200F329,66E8B1F8D3FC1497549EFC7BD645A065,0.7058200262904907,0.16101740204962262,0.45211401975677207,0.0
64153475F8549F693DC486E8E200F329,6FAB6D1BEA2D3EC0349C590A8C8AAB10,0.7058200262904907,0.16101740204962262,0.45211401975677207,0.0
64153475F8549F693DC486E8E200F329,71C705E56280CE3E496A58F7D7001CA3,0.7058200262904907,0.16101740204962262,0.45211401975677207,0.0
64153475F8549F693DC486E8E200F329,7C0541CD739A6A80B0FD1F8E095B1EF4,0.7058200262904907,0.1610174020

...

596E821BA717E768083418C5103F63E3,A660FD66872834691EBB32C35FDE44F4,0.0,0.0,0.0,0.767032080358028
596E821BA717E768083418C5103F63E3,A6C693FBFD0E49FD657328F04E902BEB,0.0,0.0,0.0,0.767032080358028
596E821BA717E768083418C5103F63E3,A7587AE69ECB24E9CE78F0E9913A4FD1,0.0,0.0,0.0,0.767032080358028
596E821BA717E768083418C5103F63E3,A8397FFF454BA93608B20285867EA26F,0.0,0.0,0.0,0.767032080358028
596E821BA717E768083418C5103F63E3,AA1E42488C399A899976B0EA74CA8193,0.0,0.0,0.0,0.767032080358028
596E821BA717E768083418C5103F63E3,AACAD841066D9B150CBD45F7C62F298D,0.0,0.0,0.0,0.767032080358028
596E821BA717E768083418C5103F63E3,ABB913F9F5512FA9E0C22C9738615EFB,0.0,0.0,0.0,0.767032080358028
596E821BA717E768083418C5103F63E3,AC940B9442BD82AE3D19243946198C9A,0.0,0.0,0.0,0.767032080358028
596E821BA717E768083418C5103F63E3,B1924096FB1C35B9835A2765ECA402D2,0.0,0.0,0.0,0.767032080358028
596E821BA717E768083418C5103F63E3,B27F056B2DBDDC0395E581BC2EC0ACE7,0.0,0.0,0.0,0.767032080358028
596E821BA717E768083418C5103F63E3,B282AAD

03791303F8D0F13929E73BCE6574944D,E084C005E3A5A79D6BF5DE4E19B7F3C0,0.4516982459268599,0.22384491433098253,0.4500910450416997,0.0
03791303F8D0F13929E73BCE6574944D,E529D7DB32A14E269266F8626B4A03FA,0.4516982459268599,0.22384491433098253,0.4500910450416997,0.0
03791303F8D0F13929E73BCE6574944D,F7D62E49B91926BE8C3C86D84D788FAC,0.4516982459268599,0.22384491433098253,0.4500910450416997,0.0
0E090862357F7E8AFA2841069792F498,09DA4DA7D44FB6193BD40137B9445240,0.2664463155743069,0.4107683145327528,0.5706448522882549,1.0
0E090862357F7E8AFA2841069792F498,334D7C6DDE9168E807C73A653FC07E9E,0.2664463155743069,0.4107683145327528,0.5706448522882549,1.0
0E090862357F7E8AFA2841069792F498,3C5A4375C44EFC3FA171C73BF4A97A16,0.5190258342335379,0.22387131146582234,0.4223524894898624,1.0
0E090862357F7E8AFA2841069792F498,D857CC5151E637638126C3E46AD86D85,0.2664463155743069,0.4107683145327528,0.5706448522882549,1.0
0E090862357F7E8AFA2841069792F498,EE81B244ED63BD7ED631398ECAD6145B,0.2664463155743069,0.4107683145327528,0.5

83F71E1A79CDFD370652FE495C517BC6,D9D014584DAB1DA3B10F4CC603C5FE48,0.0,0.0,0.0,0.0
83F71E1A79CDFD370652FE495C517BC6,DA26EA7677543A1AC5ADF4BEBDF6D4AA,0.0,0.0,0.0,0.0
83F71E1A79CDFD370652FE495C517BC6,DEB735191EC05BCAEC71BAB20302688B,0.0,0.0,0.0,0.0
83F71E1A79CDFD370652FE495C517BC6,E0AD191908DBEA29E18ECB5BF0DA1D9B,0.0,0.0,0.0,0.0
83F71E1A79CDFD370652FE495C517BC6,E7AE7EB338D247A9F1F77E16E99DBE39,0.0,0.0,0.0,0.0
83F71E1A79CDFD370652FE495C517BC6,E81BC9AC5DE1510D514343B0AFF65C67,0.0,0.0,0.0,0.0
83F71E1A79CDFD370652FE495C517BC6,E82C9FF5AE8060F259B0BBE9DA6CCF96,0.0,0.0,0.0,0.0
83F71E1A79CDFD370652FE495C517BC6,E987E926D80357417AF0EED445E35E8E,0.0,0.0,0.0,0.0
83F71E1A79CDFD370652FE495C517BC6,ED8508DA8FF00033BC0A470BDF07B70D,0.0,0.0,0.0,0.0
83F71E1A79CDFD370652FE495C517BC6,EEE6B0F0C5B84118537CFF76C18A08EE,0.0,0.0,0.0,0.0
83F71E1A79CDFD370652FE495C517BC6,EFCE2E188E83EA34C51C0923A7BB7483,0.0,0.0,0.0,0.0
83F71E1A79CDFD370652FE495C517BC6,F24F83D2782D8E2C6CAEABE28E999F44,0.0,0.0,0.0,0.0
83F71E1A79CDFD37

FFAEC223956FEE470817FEF65502ADC1,64FC00AE4C0D9F1C568513054A452768,0.5137386950437292,0.0,0.0,0.0
FFAEC223956FEE470817FEF65502ADC1,67D048ACC60D545459584546B04E0FD4,0.5137386950437292,0.0,0.0,0.0
FFAEC223956FEE470817FEF65502ADC1,67EBF147CAE9E346493906F5701530D4,0.5137386950437292,0.0,0.0,0.0
FFAEC223956FEE470817FEF65502ADC1,6D4E2344C2C09872052BCA8CE7646494,0.5137386950437292,0.0,0.0,0.0
FFAEC223956FEE470817FEF65502ADC1,6E51AF31402B013F9FE96EC8C77956B2,0.5137386950437292,0.0,0.0,0.0
FFAEC223956FEE470817FEF65502ADC1,715763B2AC655191601B041276943F95,0.5137386950437292,0.0,0.0,0.0
FFAEC223956FEE470817FEF65502ADC1,75F00B024D764EB8B73D46120B7B666A,0.5137386950437292,0.0,0.0,0.0
FFAEC223956FEE470817FEF65502ADC1,777CF1E7EFEEBE5977A5612EC254359E,0.5137386950437292,0.0,0.0,0.0
FFAEC223956FEE470817FEF65502ADC1,79BC7D3D47EF2B57B1CD456E26139352,0.5137386950437292,0.0,0.0,0.0
FFAEC223956FEE470817FEF65502ADC1,880EA3305DA18AED3A3E1487E9E9008A,0.5137386950437292,0.0,0.0,0.0
FFAEC223956FEE470817FEF65502AD

5191BC47F083EE93BE19BBC678E49863,903DD95A8F6733DBFDB2CA26D47AE2B3,0.0602397925022392,0.040474492010678514,0.14984337383656532,0.7149854423889909
5191BC47F083EE93BE19BBC678E49863,B490455E4783E75C64E3FD7DF3A4AF5F,0.0602397925022392,0.040474492010678514,0.14984337383656532,0.7149854423889909
5191BC47F083EE93BE19BBC678E49863,C533A732C5A1AF605DD747EAAC17673B,0.0602397925022392,0.040474492010678514,0.14984337383656532,0.7149854423889909
5191BC47F083EE93BE19BBC678E49863,D7D4AF947336A08BAB648E1ED55AB20C,0.0602397925022392,0.040474492010678514,0.14984337383656532,0.7149854423889909
5191BC47F083EE93BE19BBC678E49863,D87E5A4A2FD17CA7AC7DC2F8154F81EB,0.0602397925022392,0.040474492010678514,0.14984337383656532,0.7149854423889909
5191BC47F083EE93BE19BBC678E49863,DC49B1CE6BE2225C9147E836E5337E78,0.0602397925022392,0.040474492010678514,0.14984337383656532,0.7149854423889909
5191BC47F083EE93BE19BBC678E49863,EBF37B93A864F3D606F1BC0B6A157245,0.0602397925022392,0.040474492010678514,0.14984337383656532,0.71

# End NN-Model

# User-To-User Model

In [ ]:
# import Model class
%run -i "u2u.py"

In [ ]:
#train models
#imported_data = get_imported_data()

#u2u_reply = Predictor("reply", imported_data, dump_to_file=True)
#u2u_retweet = Predictor("retweet", imported_data, dump_to_file=True)
#u2u_quote = Predictor("retweet_with_comment", imported_data, dump_to_file=True)
#u2u_fav = Predictor("like", imported_data, dump_to_file=True)

In [ ]:
# load model from stored files
u2u_reply = load("model_content/u2u_reply.joblib")
u2u_retweet = load("model_content/u2u_retweet.joblib")
u2u_quote = load("model_content/u2u_retweet_with_comment.joblib")
u2u_fav = load("model_content/u2u_like.joblib")

In [ ]:
# Evaluator class that computes the f1 score. Use in evaluate_test_set!
class Evaluator:
    def __init__(self):
        self.tp = 0
        self.tn = 0
        self.fp = 0
        self.fn = 0
        
    def add(self, interaction, prediction):
        true = 0 if interaction == "" else 1
        if true == prediction and true == 0:
            self.tn += 1
        elif true == prediction and true == 1:
            self.tp += 1
        elif true != prediction and true == 0:
            self.fp += 1
        elif true != prediction and true == 1:
            self.fn += 1
            
    def get_f1(self):
        try:
            return self.tp / (self.tp + 0.5 * (self.fn + self.fp))
        except:
            return -1

In [ ]:
# User-To-User Model

path_to_data = '../../shared/data/project/validation/'
dataset_type = 'one_hour' # all_sorted, one_day, one_hour, one_week


all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager", "retweet", "reply", "like", "retweet_with_comment"]

all_features_to_idx = dict(zip(all_features, range(len(all_features))))


# adapted to also return the true interaction values
def parse_input_line(line):
    features = line #.split("\x01")
    tweet_id = features[all_features_to_idx['tweet_id']]
    user_id = features[all_features_to_idx['engaging_user_id']]
    input_feats = features[all_features_to_idx['text_tokens']]
    tweet_timestamp = features[all_features_to_idx['tweet_timestamp']]
    retweet = features[all_features_to_idx['retweet']]
    reply = features[all_features_to_idx['reply']]
    like = features[all_features_to_idx['like']]
    retweet_with_comment = features[all_features_to_idx['retweet_with_comment']]

    return tweet_id, user_id, input_feats, tweet_timestamp, retweet, reply, like, retweet_with_comment


#adapted to return the f1 values for each interaction category
def evaluate_test_set():
    expanded_path = os.path.expanduser(path_to_data)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
    
    reply_eval = Evaluator()
    retweet_eval = Evaluator()
    quote_eval = Evaluator()
    fav_eval = Evaluator()
    
        
    with open('results.csv', 'w') as output:
        for file in part_files:
            with open(file, 'r') as f:
                linereader = csv.reader(f, delimiter='\x01')
                last_timestamp = None
                for row in linereader:
                    tweet_id, user_id, features, tweet_timestamp, retweet, reply, fav, quote = parse_input_line(row)   
                    reply_pred = u2u_reply.predict(user_id, tweet_id, binary=True) # reply_model
                    retweet_pred = u2u_retweet.predict(user_id, tweet_id, binary=True) # retweet_model
                    quote_pred = u2u_quote.predict(user_id, tweet_id, binary=True) # pred_model
                    fav_pred = u2u_fav.predict(user_id, tweet_id, binary=True) # fav_model
                    
                    reply_eval.add(reply, reply_pred)
                    retweet_eval.add(retweet, retweet_pred)
                    quote_eval.add(quote, quote_pred)
                    fav_eval.add(fav, fav_pred)
                    
                    # print(str(tweet_timestamp))
                    # print(str(reply_pred)+" "+str(retweet_pred)+" "+str(quote_pred)+" "+str(fav_pred))
                    
                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')
        return reply_eval.get_f1(), retweet_eval.get_f1(), quote_eval.get_f1(), fav_eval.get_f1()




In [ ]:
reply_eval, retweet_eval, quote_eval, fav_eval = evaluate_test_set()

print(f"F1-Reply: {reply_eval}")
print(f"F1-Retweet: {reply_tweet}")
print(f"F1-Quote: {reply_quote}")
print(f"F1-Like: {reply_fav}")

# -- End User-To-User Model --

In [ ]:

data = import_data(path_to_data+dataset_type)
data

In [ ]:
print(" -- unique tweets --")
print(data["tweet_id"].unique().size)
print(" -- engaged users --")
print(data["engaged_with_user_id"].unique().size)
print(" -- engaging users --")
print(data["engaging_user_id"].unique().size)

In [ ]:
# hidden


# Metrics Functions



## Recsys Challenge Functions

In [ ]:
from sklearn.metrics import average_precision_score, log_loss

def calculate_ctr(gt):
  positive = len([x for x in gt if x == 1])
  ctr = positive/float(len(gt))
  return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

  
ground_truth = read_predictions(path_to_data + dataset_type) # will return data in the form (tweet_id, user_id, labed (1 or 0))
predictions = read_predictions("predictions.csv") # will return data in the form (tweet_id, user_id, prediction)

rce = compute_rce(predictions, ground_truth)
average_precision = average_precision_score(ground_truth, predictions)

print(rce)
print(average_precision)

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit